# Uncover ML target preprocessoor
#### This notebook exists to perform statistical preanalysis on input geochemistry target files for the UncoverML project
* Import a .csv file containing geochemistry data with LAT LON
* Process negative detection limits to useable data
* Group the data by duplicated measurements taken at each location
* Calculate the median of those groups and replace each group with the median value in the original dataframe
* Remove outliers

## Data import and header / value standardisation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from numbers import Number

In [ ]:
# Read in geochemistry data

ozchem = pd.read_csv("champ_chem_rock_rego_geo_mapV3.csv", header=0, low_memory=False)
nt = pd.read_csv("nt_gchem_v2.csv", header=0, low_memory=False)
qld = pd.read_csv("qld_reprojected.csv", header=0, low_memory=False)
wa = pd.read_csv("wa_gchem_v2.1.csv", header=0, low_memory=False)
nsw = pd.read_csv("nsw_shifted.csv", header=0, low_memory=False)
sa = pd.read_csv("sa_gchem_v2.csv", header=0, low_memory=False)
sa_calc = pd.read_csv("calcrete_geology_SA_v1.csv", header=0, low_memory=False)
sa_rock = pd.read_csv("SA_rock_ppm.csv", header=0, low_memory=False)

regolith = pd.read_csv("regolithmaster.csv", header=0, low_memory=False)

In [ ]:
# Assign a name identifier before merge

ozchem['name'] = 'ozchem'
nt['name'] = 'nt'
qld['name'] = 'qld'
wa['name'] = 'wa'
nsw['name'] = 'nsw'
sa['name'] = 'sa'
sa_calc['name'] = 'sa_calc'
sa_rock['name'] = 'sa_rock'
regolith['name'] = 'regolith'

In [ ]:
# These print statements exist to show all the elements in each input dataset to create the element list

# print(list(ozchem))
# print(list(nt))
# print(list(qld))
# print(list(wa))
# print(list(nsw))
# print(list(sa))
# print(list(sa_calc))
# print(list(sa_rock))
# print(list(regolith))

In [ ]:
# Calculate the oxides in the sa_rock and sa_calc datasets and add them back into their respective dataframes

def al_oxide(al):
    return al/0.5293

def ca_oxide(ca):
    return ca/0.7147

def k_oxide(k):
    return k/0.8301

def mg_oxide(mg):
    return mg/0.603

def na_oxide(na):
    return na/0.7419

def si_oxide(si):
    return si/0.4674

def fe_oxide(fe):
    return fe/0.6994

sa_rock_elements = ['Fe__', 'Mg__', 'Ca__', 'Si__', 'Na__', 'K__', 'Al__']
sa_calc_elements = ['Ca_pct', 'Fe_pct','Mg_pct','K_pct']

sa_calc['fe2o3_pct'] = np.vectorize(fe_oxide)(sa_calc['Fe_pct'])
sa_calc['cao_pct'] = np.vectorize(ca_oxide)(sa_calc['Ca_pct'])
sa_calc['k2o_pct'] = np.vectorize(k_oxide)(sa_calc['K_pct'])
sa_calc['mgo_pct'] = np.vectorize(mg_oxide)(sa_calc['Mg_pct'])
sa_rock['fe2o3_pct'] = np.vectorize(fe_oxide)(sa_rock['Fe__']/10000)
sa_rock['cao_pct'] = np.vectorize(ca_oxide)(sa_rock['Ca__']/10000)
sa_rock['k2o_pct'] = np.vectorize(k_oxide)(sa_rock['K__']/10000)
sa_rock['mgo_pct'] = np.vectorize(mg_oxide)(sa_rock['Mg__']/10000)
sa_rock['sio2_pct'] = np.vectorize(ca_oxide)(sa_rock['Si__']/10000)
sa_rock['na2o_pct'] = np.vectorize(k_oxide)(sa_rock['Na__']/10000)
sa_rock['al2o3_pct'] = np.vectorize(mg_oxide)(sa_rock['Al__']/10000)


**Modify the column headings of the dataframe for consistency**

**Create a generic list of elements by which to conduct summary statistics on**

**Calculate the intersection of columns and element lists**

In [ ]:
# Set the inputs for the merge loop

inputs = [ozchem, nt, qld, wa, nsw, sa, sa_calc, sa_rock, regolith]

In [ ]:
national_gchem = pd.DataFrame(columns=[], dtype=float)

for df in inputs:
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    df = df.replace([-9999, -999, 99999], np.nan)
    columns = list(df.columns)
    location = ['lon', 'long', 'lat', 'longitude', 'latitude', 'longda94', 'latgda94', 'dlat','dlong']
    location_in_columns = list(set.intersection(set(location), set(columns)))
    fe2o3 = ['fe2o3_pct','fe2o3', 'fe2o3_pc']
    fe2o3t = ['fe2o3t_pct', 'fe2o3t', 'fe2o3tot']
    feo = ['feo_pct','feo', 'feo_pc']
    al = ['al2o3', 'al2o3_pc', 'al2o3_pct']
    si = ['sio2', 'sio2_pct', 'sio2_pc']
    ca = ['cao', 'cao_pct', 'cao_pc']
    mg = ['mgo', 'mgo_pct', 'mgo_pc']
    na = ['na2o', 'na2o_pct', 'na2o_pc']
    k = ['k2o', 'k2o_pct', 'k2o_pc']
    ti = ['tio2', 'tio2_pct', 'tio2_pc']
    mn = ['mno', 'mno_pct', 'mno_pc']
    p = ['p2o5', 'p2o5_pct', 'p2o5_pc']
    fe2o3t_in_columns = list(set.intersection(set(fe2o3t), set(columns)))
    fe2o3_in_columns = list(set.intersection(set(fe2o3), set(columns)))
    feo_in_columns = list(set.intersection(set(feo), set(columns)))
    al_in_columns = list(set.intersection(set(al), set(columns)))
    si_in_columns = list(set.intersection(set(si), set(columns)))
    ca_in_columns = list(set.intersection(set(ca), set(columns)))
    mg_in_columns = list(set.intersection(set(mg), set(columns)))
    na_in_columns = list(set.intersection(set(na), set(columns)))
    k_in_columns = list(set.intersection(set(k), set(columns)))
    ti_in_columns = list(set.intersection(set(ti), set(columns)))
    mn_in_columns = list(set.intersection(set(mn), set(columns)))
    p_in_columns = list(set.intersection(set(p), set(columns)))
    keep = location_in_columns + fe2o3_in_columns + feo_in_columns + fe2o3t_in_columns + al_in_columns + si_in_columns + ca_in_columns + mg_in_columns + na_in_columns + k_in_columns + ti_in_columns + p_in_columns + mn_in_columns + ['name'] + ['sample_no'] + ['sampleid'] + ['sample_id'] + [
        'lithology'] + ['reliability'] + ['lithname'] + ['lith_group'] + ['rock_type'] + ['collected_date'] + ['extraction_date'] + ['send_date'] + ['collected1']
    df = df.filter(keep)
    df.rename(columns={'long': 'longitude','lon': 'longitude', 'lat': 'latitude', 'longda94': 'longitude',
                       'latgda94': 'latitude','dlat': 'latitude', 'dlong': 'longitude', 'extraction_date': 'date', 'collected_date':'date', 'collected1': 'date', 'send_date':'date', 'fe2o3': 'fe2o3_pct', 'fe2o3_pc':'fe2o3_pct','feo':'feo_pct','feo_pc': 'feo_pct', 'fe2o3t':'fe2o3t_pct', 'fe2o3tot':'fe2o3t_pct', 'al2o3': 'al2o3_pct', 'al2o3_pc':'al2o3_pct', 'sio2': 'sio2_pct', 'sio2_pc': 'sio2_pct', 'cao' : 'cao_pct', 'cao_pc' : 'cao_pct', 'mgo': 'mgo_pct', 'mgo_pc':'mgo_pct', 'na2o': 'na2o_pct', 'na2o_pc':'na2o_pct', 'k2o':'k2o_pct', 'k2o_pc':'k2o_pct', 'tio2': 'tio2_pct', 'tio2_pc':'tio2_pct', 'mno':'mno_pct', 'mno_pc':'mno_pct', 'p2o5': 'p2o5_pct', 'p2o5_pc':'p2o5_pct', 'sampleid': 'sample_id'}, inplace =True)
    df.sort_index(axis=1, inplace=True)
    national_gchem = pd.concat([national_gchem, df], axis=0)
    print(df.name[1])

In [ ]:
# set list of elements to be present in the final dataset

elements = ['al2o3_pct','feo_pct','fe2o3_pct', 'fe2o3t_pct', 'sio2_pct', 'mgo_pct', 'cao_pct', 'na2o_pct', 'k2o_pct', 'tio2_pct', 'p2o5_pct', 'mno_pct']

In [ ]:
# Coerce all the elements and the location data to numeric type

national_gchem[elements] = national_gchem[elements].apply(pd.to_numeric, errors='coerce')
national_gchem[['latitude', 'longitude']] = national_gchem[['latitude', 'longitude']].apply(pd.to_numeric, errors='coerce')

In [ ]:
# Remove rows where all elements have a value of NaN or 0
print('number of observations:', national_gchem.shape[0])
national_gchem.dropna(subset = elements, how='all', inplace=True)
print('number of observations with NaN rows removed:', national_gchem.shape[0])
national_gchem = national_gchem[~(national_gchem[elements] == 0).all(axis = 1)]
print('number of observations with 0 rows removed:', national_gchem.shape[0])

## Duplicate, outlier and detection limit processing of geochemistry measurements

In [ ]:
# Convert detection limits in a set range to useful vales by dividing by -2
# Convert values with 0 to nan and those less than -2.5 (the lower detection limit) to nan

def detectionlimits(value):
    if -2.5 < value < 0:
        modified_value = value / -2
        return modified_value
    else:
        return value

vdetectionlimits = np.vectorize(detectionlimits)

for column in national_gchem[elements]:
    national_gchem.loc[national_gchem[column] <= -2.5, column] = np.nan
    national_gchem.loc[national_gchem[column] == 0, column] = np.nan


national_gchem[elements] = national_gchem[elements].apply(vdetectionlimits)

In [ ]:
# (Re)Calculate total iron under 'fe2o3t_pct

# Function to check whether a number is a float or integer and not a NaN.

def numberchecker(value):
    if isinstance(value, Number) and pd.isnull(value) == False and value != 0:
        return True
    else:
        return False

# Keep the existing fe2o3t measurement if it's not similar to either of its inputs
# Calculate the total iron if both fe2o3 and feo are numbers    

def totalfe(fe2o3, feo, fe2o3t):
    if  numberchecker(fe2o3t) and not (fe2o3-.02) < fe2o3t < (fe2o3+.02) and not (feo-.02) < fe2o3t < (feo+.02):
        return fe2o3t
    elif numberchecker(fe2o3) and numberchecker(feo):
        return fe2o3 + feo*1.11134
    elif numberchecker(fe2o3):
        return fe2o3
    elif numberchecker(feo):
        return feo*1.11134
    else:
        return None

national_gchem['fe2o3t_pct'] = np.vectorize(totalfe)(national_gchem['fe2o3_pct'], national_gchem['feo_pct'], national_gchem['fe2o3t_pct'])
elements = ['al2o3_pct','feo_pct', 'fe2o3t_pct','fe2o3_pct', 'sio2_pct', 'mgo_pct', 'cao_pct', 'na2o_pct', 'k2o_pct', 'tio2_pct', 'p2o5_pct', 'mno_pct']

In [ ]:
# Calculate median of duplicate measurements at locations
# Recalculate the list of elements incase empty columns were dropped in the groupby

national_gchem_dupemedian = national_gchem.groupby(['longitude', 'latitude'], as_index=False)[elements].agg('median')
columns = list(national_gchem_dupemedian.columns)
elements = list(set.intersection(set(elements), set(columns)))
print('shape before calculating median of duplicates at locations:', national_gchem.shape)
print('shape after calculating median of duplicates at locations:', national_gchem_dupemedian.shape)

In [ ]:
# Make observations of samples with over 100% or over the 99th percentile of a single element equal to NaN

national_gchem_dupemedian_outlier = national_gchem_dupemedian.copy()

for column in national_gchem_dupemedian_outlier[elements]:
    max = np.nanpercentile(national_gchem_dupemedian_outlier[column], 99)
    national_gchem_dupemedian_outlier.loc[national_gchem_dupemedian_outlier[column] > 100, column] = np.nan
    national_gchem_dupemedian_outlier.loc[national_gchem_dupemedian_outlier[column] > max, column] = np.nan
    
# Drop rows where all values are equal to NaN (removing the outliers defined above')

print('Before outliers removed:', national_gchem_dupemedian_outlier.shape)
national_gchem_dupemedian_outlier.dropna(subset = elements, inplace=True, how='all')
print('After outliers removed:', national_gchem_dupemedian_outlier.shape)

## Merging datasets


In [ ]:
def rmerge(left,right,**kwargs):

    # Function to flatten lists from http://rosettacode.org/wiki/Flatten_a_list#Python
    def flatten(lst):
        return sum( ([x] if not isinstance(x, list) else flatten(x) for x in lst), [] )
    
    # Set default for removing overlapping columns in "left" to be true
    myargs = {'replace':'left'}
    myargs.update(kwargs)
    
    # Remove the replace key from the argument dict to be sent to
    # pandas merge command
    kwargs = {k:v for k,v in myargs.items() if k is not 'replace'}
    
    if myargs['replace'] is not None:
        # Generate a list of overlapping column names not associated with the join
        skipcols = set(flatten([v for k, v in myargs.items() if k in ['on','left_on','right_on']]))
        leftcols = set(left.columns)
        rightcols = set(right.columns)
        dropcols = list((leftcols & rightcols).difference(skipcols))
        
        # Remove the overlapping column names from the appropriate DataFrame
        if myargs['replace'].lower() == 'left':
            left = left.copy().drop(dropcols,axis=1)
        elif myargs['replace'].lower() == 'right':
            right = right.copy().drop(dropcols,axis=1)
        
    df = pd.merge(left,right,**kwargs)
    
    return df

**Drop the duplicates from the original dataset,  keeping the first observation at a location**

**Fill the geochemistry values in the above dataset with the different levels of processed data whilst keeping the metadata using the defined rmerge function**

In [ ]:
# drop duplicates based on lat and lon
national_gchem_dupe_drop = national_gchem.drop_duplicates(
    ['longitude', 'latitude'], keep='first'
)

# replace the geochemistry values in the duplicate dropped dataset with the declim_dupemedian values
national_gchem_dupemedian = rmerge(
    national_gchem_dupe_drop, national_gchem_dupemedian, on=['longitude','latitude']
)

# add metadata back to duplicate median dataset
national_gchem_dupemedian_outlier = rmerge(
    national_gchem_dupe_drop, national_gchem_dupemedian_outlier, on=['longitude','latitude']
)


In [ ]:
print(national_gchem.shape)
print(national_gchem_dupe_drop.shape)
print(national_gchem_dupemedian.shape)
print(national_gchem_dupemedian_outlier.shape)


In [ ]:
# Extract the year from the input data and add is as a column

def yearfinder(year):
    if pd.isnull(year):
        return year
    x = re.findall('[\d]{4}', year)
    #result = [int(x) for x in x]
    return x

national_gchem_dupemedian_outlier['year'] = national_gchem_dupemedian_outlier['date'].apply(yearfinder)

In [ ]:
'''This list is needed to drop duplicate entries based on their geochemistry values 
excluding total iron because of slight variations in it's calculation'''

elements_no_fe2o3t = ['al2o3_pct','feo_pct','fe2o3_pct', 'sio2_pct', 'mgo_pct', 'cao_pct', 'na2o_pct', 'k2o_pct', 'tio2_pct', 'p2o5_pct', 'mno_pct']

In [ ]:
# Create a list of tuples that contain the index pairs of rows with identical elements

# element_duplicates = national_gchem_dupe_drop[national_gchem_dupe_drop[elements_no_fe2o3t].duplicated(keep=False)]
# element_duplicates = element_duplicates.groupby(element_duplicates[elements_no_fe2o3t].columns.tolist()).apply(lambda x: tuple(x.index)).tolist()
# element_duplicates = pd.DataFrame(element_duplicates, columns = ['element_duplicate_index_1', 'element_duplicate_index_2', 'element_duplicate_index_3', 'element_duplicate_index_4'])
# element_duplicates.drop(columns = ['element_duplicate_index_3', 'element_duplicate_index_4'], inplace=True)

In [ ]:
national_gchem_dupemedian_outlier['FID'] = range(0, len(national_gchem_dupemedian_outlier))

In [ ]:
# national_gchem_dupemedian_outlier = pd.merge(
#     national_gchem_dupemedian_outlier, element_duplicates, how='left', left_on = 'FID', right_on = 'element_duplicate_index_2')

In [ ]:
national_gchem_dupemedian_outlier.head()

In [ ]:
national_gchem_dupemedian_outlier.to_csv('final.csv', sep = ',', index=False)
# 

# Geospatial processing

In [ ]:
national_gchem_dupemedian_outlier = pd.read_csv('test.csv', header=0, low_memory=False)

In [ ]:
import sys
import geopandas as gpd
from scipy.spatial import cKDTree
from shapely.geometry import Point
from fiona.crs import from_epsg
sys.path.append('/g/data/u46/users/sc0554/anaconda3/envs/work')
sys.path.append('/g/data/u46/users/sc0554/anaconda3/lib')


Convert the dataframe to a geodataframe by extracting the geometry from the Lat and Lon and projecting it to Albers

In [ ]:
geometry = [Point(xy) for xy in zip(national_gchem_dupemedian_outlier.longitude, 
                                    national_gchem_dupemedian_outlier.latitude)]

#national_gchem_declim_dupemedian_outlier_elementdrop = national_gchem_declim_dupemedian_outlier_elementdrop.drop(['longitude', 'latitude'], axis=1)
crs = {'init': 'epsg:4326'}
national_gchem_shape = gpd.GeoDataFrame(national_gchem_dupemedian_outlier, crs=crs, geometry=geometry)
national_gchem_shape.drop(columns=['latitude', 'longitude'], inplace=True)
national_gchem_shape['geometry'] = national_gchem_shape['geometry'].to_crs(epsg = 3577)
national_gchem_shape.crs = from_epsg(3577)
national_gchem_shape.crs


Add geometry as x, y columns in dataframe

In [ ]:
def getXY(pt):
    return (pt.x, pt.y)
centroidseries = national_gchem_shape['geometry'].centroid
x,y = [list(t) for t in zip(*map(getXY, centroidseries))]
national_gchem_shape['x'] = x
national_gchem_shape['y'] = y

Calculate the spatial distance between the observations with element duplicates

In [ ]:
# Function that creates a list of tuples that contain the index pairs of rows with identical elements

def duplicate_distance_calculator(df, dupe_list):
    index_1_list = []
    index_2_list = []
    fid_1_list = []
    fid_2_list = []
    distances = []
    #mark all duplicates using the input list as True
    duplicates = df[df[dupe_list].duplicated(keep=False)]
    #conduct a groupby on just the duplicates to find the indexes of each duplicate pair
    duplicates = duplicates.groupby(duplicates[dupe_list].columns.tolist()).apply(lambda x: tuple(x.index)).tolist()
    for duplicate_pair in duplicates:
        index_1 = duplicate_pair[0]
        index_2 = duplicate_pair[1]
        duplicate_1_location = df.iloc[index_1].geometry
        duplicate_1_fid = df.iloc[index_1].FID
        duplicate_2_location = df.iloc[index_2].geometry
        duplicate_2_fid = df.iloc[index_2].FID
        #calculate the distance between the two points that are duplicates
        distance = duplicate_1_location.distance(duplicate_2_location)
        #append this to a list
        fid_1_list.append(duplicate_1_fid)
        fid_2_list.append(duplicate_2_fid)
        index_1_list.append(index_1)
        index_2_list.append(index_2)
        distances.append(distance)
    distance_indexes = pd.DataFrame({'index_1': fid_1_list, 'index_2': fid_2_list, 'geochem_dupe_distance': distances})
    return distance_indexes

In [ ]:
distanceindexes = duplicate_distance_calculator(national_gchem_shape, elements)

In [ ]:
# Add the duplicate indexes and distances to the main geodataframe

national_gchem_shape_distances = pd.merge(national_gchem_shape, distanceindexes, how='left', left_on='FID', right_on='index_1')
#national_gchem_shape_distances = pd.merge(national_gchem_shape_distances, distanceindexes, how='left', left_on='FID', right_on='index_2')
national_gchem_shape_distances.update(distanceindexes[['index_1', 'index_2', 'geochem_dupe_distance']])

In [ ]:
# national_gchem_shape_distances = pd.read_csv('nat_gchem_distances2.csv')

In [ ]:
national_gchem_shape_distances.head()

In [ ]:
# make a list of the FIDs to drop from the dataframe based on their distance, preferencing ozchem
indexes_to_drop = []
for duplicates in distanceindexes.itertuples():
    if duplicates.geochem_dupe_distance < 40:
        index_1 = duplicates.index_1
        index_2 = duplicates.index_2
        duplicate_1 = national_gchem_shape_distances[national_gchem_shape_distances['FID'] == index_1]
        duplicate_2 = national_gchem_shape_distances[national_gchem_shape_distances['FID'] == index_2]
        if duplicate_1.name.item() != 'ozchem':
            indexes_to_drop.append(duplicate_1.FID.item())
        if duplicate_2.name.item() !='ozchem':
            indexes_to_drop.append(duplicate_2.FID.item())
        else:
            indexes_to_drop.append(duplicate_1.FID.item())
    if duplicates.geochem_dupe_distance > 40:
        indexes_to_drop.append(duplicates.index_1)
        indexes_to_drop.append(duplicates.index_2)

In [ ]:
national_gchem_shape_distances_dropped = national_gchem_shape_distances.loc[
    ~national_gchem_shape_distances['FID'].isin(indexes_to_drop)]

print(national_gchem_shape_distances.shape)
print(national_gchem_shape_distances_dropped.shape)

In [ ]:
print('# of observations', national_gchem_shape_distances.shape[0])
print('# of observations where the duplicate element pairs are greater than 40 meters apart:', national_gchem_shape_distances.loc[
    national_gchem_shape_distances.geochem_dupe_distance > 40].shape[0])
print('# of observations where the duplicate element pairs are less than 40 meters apart:', national_gchem_shape_distances.loc[
    national_gchem_shape_distances.geochem_dupe_distance < 40].shape[0])
print('# of observations after dropping:', national_gchem_shape_distances_dropped.shape[0])

In [ ]:
pd.set_option('precision', 2)


In [ ]:
print("Preprocessed elements")
national_gchem[elements].round(1).describe()

In [ ]:
print("Post-processed elements")
national_gchem_shape_distances[elements].round(1).describe()

In [ ]:
# Plot of the frequency of distances between duplicate pairs

distanceindexes['geochem_dupe_distance'].loc[distanceindexes['geochem_dupe_distance'] < 100].plot(kind= 'hist', bins=20, log=True)

In [ ]:
plot_elements = ['al2o3_pct','feo_pct','fe2o3_pct', 'mgo_pct', 'cao_pct', 'sio2_pct', 'na2o_pct', 'k2o_pct', 'tio2_pct', 'p2o5_pct', 'mno_pct']
national_gchem[plot_elements].plot(kind = 'box', vert=False, figsize =(20,5), xlim=(0,100))
plt.show()

In [ ]:
national_gchem_shape_distances[plot_elements].plot(kind = 'box', vert=False, figsize =(20,5), xlim=(0,100))
plt.show()

In [ ]:
national_gchem_shape_distances.to_csv('nat_gchem_distances_final.csv', sep = ',', index=False)

# national_gchem_shape_distances_dropped.to_csv('nat_gchem_distances_dropped_final.csv', sep = ',', index=False)


In [ ]:
national_gchem_shape_distances.to_file('national_gchem.shp')
national_gchem_shape_distances_dropped.to_file('national_gchem_dropped.shp')